<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Portfolio Allocation

Tutorials to use OpenAI DRL to perform portfolio allocation in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd
* Please report any issues to our Github: https://github.com/AI4Finance-Foundation/FinRL/issues
* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [144]:
## install finrl library
!pip install wrds
!pip install swig

!pip install 'shimmy>=2.0'
!pip install pandas_market_calendars
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

zsh:1: command not found: apt-get
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-ysz_2axk
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-ysz_2axk
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 69776b349ee4e63efe3826f318aef8e5c5f59648
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-mfeqwwlq/elegantrl_b5a933463c3144fc8ca448da9be2814e
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-mfeqwwlq/elegantrl_b


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [145]:
# ===========================
# Suppress Warnings & Backend Setup
# ===========================
import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for environments without GUI

# ===========================
# Standard Libraries
# ===========================
import os
import sys
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For Jupyter Notebooks (optional, safe to keep)
%matplotlib inline

# ===========================
# FinRL Imports
# ===========================
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from pypfopt.efficient_frontier import EfficientFrontier


# ===========================
# Custom Library Path
# ===========================
sys.path.append("../FinRL-Library")

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [146]:
def process_csv_to_features(csv_path, lookback=252):
    # Step 1: Load data
    df = pd.read_csv(csv_path)

    # Step 2: Identify 5-day and 7-day tickers
    day_values_per_tic = df.groupby('tic')['day'].apply(lambda x: sorted(x.unique())).reset_index()
    day_values_per_tic.columns = ['tic', 'unique_days']
    tics_5day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(5)))]['tic']
    tics_7day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(7)))]['tic']

    # Step 3: Filter tickers
    df_5day_full = df[df['tic'].isin(tics_5day)]
    df_7day_full = df[df['tic'].isin(tics_7day)]

    # Step 4: Apply technical indicators
    fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
    df_5day_full = fe.preprocess_data(df_5day_full)
    if not df_7day_full.empty:
        df_7day_full = fe.preprocess_data(df_7day_full)
    else:
        print("[Info] df_7day_full is empty. Skipping technical indicators.")

    # Step 5: Combine and clean
    df = pd.concat([df_5day_full, df_7day_full], ignore_index=False)
    df.index = range(len(df))
    df['date'] = pd.to_datetime(df['date'])
    df = df[df.groupby('date')['date'].transform('count') > 1]
    df = df.sort_values('date').reset_index(drop=True)

    # Step 6: Prepare for covariance matrix computation
    df = df.sort_values(['date', 'tic'], ignore_index=True)
    df.index = df.date.factorize()[0]  # Re-index based on unique date

    cov_list = []
    return_list = []
    unique_indices = df.index.unique()

    for i in range(lookback, len(unique_indices)):
        data_lookback = df.loc[i - lookback:i, :]
        price_lookback = data_lookback.pivot_table(index='date', columns='tic', values='close')
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)
        cov_list.append(return_lookback.cov().values)

    # Step 7: Merge covariance matrix and return series back
    df_cov = pd.DataFrame({
        'date': df.date.unique()[lookback:], 
        'cov_list': cov_list, 
        'return_list': return_list
    })
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date', 'tic']).reset_index(drop=True)

    return df


In [147]:
processed_0 = process_csv_to_features('2007-2025_no_crypto.csv')
processed_1 = process_csv_to_features('2015-2025_crypto.csv')
processed_2 = process_csv_to_features('2015-2025_no_crypto.csv')

Successfully added technical indicators
[Info] df_7day_full is empty. Skipping technical indicators.
Successfully added technical indicators
Successfully added technical indicators
Successfully added technical indicators
[Info] df_7day_full is empty. Skipping technical indicators.


## Training data split: 2009-01-01 to 2020-07-01

In [148]:
import os
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier

def run_min_variance_portfolio(df, 
                                start_date, 
                                end_date, 
                                initial_capital=1_000_000, 
                                weight_bound=(0, 0.15),
                                output_return_csv='df_daily_return_min.csv',
                                original_csv_path='data.csv'):
    """
    Calculate Minimum Variance Portfolio and organize outputs in /<csv_name>/<model_name>/ structure.
    """

    # === Step 1: Setup Folder Structure ===
    base_name = os.path.splitext(os.path.basename(original_csv_path))[0]   # e.g., 'data'
    model_folder = "min"
    target_folder = os.path.join(base_name, model_folder)

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"[INFO] Created folder: {target_folder}")

    # Adjust CSV path to save inside the folder
    output_csv_full_path = os.path.join(target_folder, output_return_csv)

    # === Step 2: Filter trade dates ===
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    
    trade = data_split(df, TRADE_START_DATE, TRADE_END_DATE)
    unique_trade_date = trade.date.unique()

    trade_dates = [d for d in unique_trade_date if start_date <= d <= end_date]

    if len(trade_dates) < 2:
        raise ValueError("Not enough trade dates between start_date and end_date.")

    # === Step 3: Initialize Portfolio ===
    portfolio = pd.DataFrame(index=range(1), columns=trade_dates)
    portfolio.loc[0, trade_dates[0]] = initial_capital

    # === Step 4: Loop Through Dates for Min Variance Allocation ===
    for i in range(len(trade_dates) - 1):
        df_temp = df[df.date == trade_dates[i]].reset_index(drop=True)
        df_temp_next = df[df.date == trade_dates[i + 1]].reset_index(drop=True)

        Sigma = df_temp.return_list[0].cov()

        ef_min_var = EfficientFrontier(None, Sigma, weight_bounds=weight_bound)
        ef_min_var.min_volatility()
        cleaned_weights = ef_min_var.clean_weights()

        cap = portfolio.iloc[0, i]
        current_cash = [w * cap for w in cleaned_weights.values()]
        current_shares = np.array(current_cash) / np.array(df_temp.close)
        next_price = np.array(df_temp_next.close)

        portfolio.iloc[0, i + 1] = np.dot(current_shares, next_price)

    # === Step 5: Calculate Daily Returns ===
    portfolio = portfolio.T
    portfolio.columns = ['account_value']

    df_daily_return = portfolio.copy()
    df_daily_return["daily_return"] = df_daily_return["account_value"].pct_change()
    df_daily_return = df_daily_return.infer_objects(copy=False)
    df_daily_return = df_daily_return.reset_index().rename(columns={"index": "date"})
    df_daily_return.loc[0, "daily_return"] = 0.0
    df_daily_return = df_daily_return[["date", "daily_return"]]

    # === Step 6: Save to CSV in Target Folder ===
    df_daily_return.to_csv(output_csv_full_path, index=False)
    print(f"[INFO] Daily returns saved to {output_csv_full_path}")

    return df_daily_return


In [149]:
# df_daily_return_min = run_min_variance_portfolio(
#     df=processed,
TRADE_START_DATE = '2023-04-05'
TRADE_END_DATE = '2025-04-10'


df_daily_return_min = run_min_variance_portfolio(
    df=processed_0,
    start_date=TRADE_START_DATE,
    end_date=TRADE_END_DATE,
    initial_capital=1_000_000,
    weight_bound=(0, 0.15),
    output_return_csv='df_daily_return_min.csv',
    original_csv_path='2007-2025_no_crypto.csv'
)

df_daily_return_min = run_min_variance_portfolio(
    df=processed_1,
    start_date=TRADE_START_DATE,
    end_date=TRADE_END_DATE,
    initial_capital=1_000_000,
    weight_bound=(0, 0.15),
    output_return_csv='df_daily_return_min.csv',
    original_csv_path='2015-2025_crypto.csv'
)

df_daily_return_min = run_min_variance_portfolio(
    df=processed_2,
    start_date=TRADE_START_DATE,
    end_date=TRADE_END_DATE,
    initial_capital=1_000_000,
    weight_bound=(0, 0.15),
    output_return_csv='df_daily_return_min.csv',
    original_csv_path='2015-2025_no_crypto.csv'
)

[INFO] Daily returns saved to 2007-2025_no_crypto/min/df_daily_return_min.csv
[INFO] Daily returns saved to 2015-2025_crypto/min/df_daily_return_min.csv
[INFO] Daily returns saved to 2015-2025_no_crypto/min/df_daily_return_min.csv
